# Individual Scores

- Download files containing losses and f1 scores for each of the tasks trained separately
- Parse out this information
- Plot learning curves

### Current Running Jobs

In [ ]:
!ssh submit64.cs.utexas.edu 'condor_q ebanner'

### Get Files

In [ ]:
%store -r pruned_dataset

files = pruned_dataset.columns.tolist()

for file in files:
    !rm -rf /tmp/$file

    !scp -r submit64.cs.utexas.edu:scratch/code/cochrane-nlp/experiments/ct.gov/cnn/output/$file /tmp/

### Filter Away Empty Directories

In [ ]:
import os

files = [file for file in files if os.listdir('/tmp/{}'.format(file))]

### Combine Separate Runs into One File

In [ ]:
for file in files:
    outfiles = !ls /tmp/$file/out.*

    for outfile in sorted(outfiles):
        !cat "$outfile" >> /tmp/$file/out

### Filter Away Empty Files

In [ ]:
import os

files = [file for file in files if os.path.isfile('/tmp/{}/out'.format(file)) and os.stat('/tmp/{}/out'.format(file)).st_size]

files

### Aggregate Scores

In [ ]:
from collections import OrderedDict

def performance_generator(targets):
    """Generate performance for each specified target
    
    Performance includes history of train and validation loss, as well as f1 scores.
    
    """
    for target in targets:
        val_losses = !grep 'val loss' /tmp/$target/out | cut -d' ' -f 3
        val_losses = [float(val_loss) for val_loss in val_losses]

        train_losses = !grep -- '- loss' /tmp/$target/out | cut -d' ' -f 4
        train_losses = [float(train_loss) for train_loss in train_losses]

        f1s = !grep 'f1' /tmp/$target/out | cut -d' ' -f 3
        f1s = [float(f1) for f1 in f1s]
        
        yield target, OrderedDict(train_losses=train_losses, f1s=f1s, val_losses=val_losses)

### Best Scores

In [ ]:
def best_generator():
    """Yields the best f1 score for each class"""
    
    for target, score_dict in performance_generator(files):
        yield target, [np.max(score_dict['f1s'])]
        
besties = dict(best_generator())
               
if besties:
    pd.DataFrame(besties, index=['best_f1']).T.plot(kind='bar')

### Plot Learning Curves

- Blue - Train loss
- Red - Validation loss
- Green - f1

In [ ]:
performances = performance_generator(files)

num_plots = len(files)

fig = plt.figure(figsize=(16, num_plots*2))
plt.clf()

for i, (target, score_dict) in enumerate(performances, start=1):
    
    axes = fig.add_subplot((num_plots+1)/2, 3, i)

    df = pd.DataFrame(score_dict)
    df.index.name = 'epochs'
    
    df.plot(ax=axes, title=target, legend=False)
    
fig.subplots_adjust(wspace=.35, hspace=.65)